In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai import *
import warnings
warnings.filterwarnings('ignore')

### Data

In [ ]:
path = Path('D:\College\BE\BEPROJECT\Dataset2')

In [ ]:
np.random.seed(41)
data = ImageDataBunch.from_folder(path, valid_pct = 0.2 , size=(256,256), bs=64,ds_tfms = get_transforms() ,num_workers=4).normalize()

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
print("Number of examples in training:", len(data.train_ds))
print("Number of examples in validation:", len(data.valid_ds))

In [ ]:
xb,yb = data.one_batch()
xb.shape,yb.shape

In [ ]:
#Sample images from the dataset
data.show_batch(rows=3, figsize=(10,10))

## DarkCovidNet (modified Darknet model)

In [ ]:
def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

In [ ]:
def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    )

In [ ]:
def maxpooling():
    return nn.MaxPool2d(2, stride=2)

In [ ]:
model = nn.Sequential(
    conv_block(3, 8),
    maxpooling(),
    conv_block(8, 16),
    maxpooling(),
    triple_conv(16, 32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_layer(256, 2),
    Flatten(),
    nn.Linear(338, 2),
    nn.Softmax(dim = 1)
)

In [ ]:
learn = Learner(data, model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)

In [ ]:
print(learn.summary())

In [ ]:
lr_find(learn)

In [ ]:
learn.recorder.plot(suggestion = True)

In [ ]:
learn.fit_one_cycle(50, max_lr=1e-02)

In [ ]:
print("Number of examples in testing:", len(data.valid_ds))

In [ ]:
probs,targets = learn.get_preds(ds_type=DatasetType.Valid) 

In [ ]:
accuracy(probs,targets)

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
probs = np.argmax(probs, axis=1)
correct = 0
for idx, pred in enumerate(probs):
    if pred == targets[idx]:
        correct += 1
accuracy = correct / len(probs)
print(len(probs), correct, accuracy*100)


from sklearn.metrics import classification_report
y_true1 = targets
y_pred1 = probs
target_names = ['Covid-19', 'No_findings']
print(classification_report(y_true1, y_pred1, target_names=target_names))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
cm_fig = interp.plot_confusion_matrix(return_fig=True)

In [ ]:
learn.export('D:\College\BE\BEPROJECT\Dataset2\model')

In [ ]:
from fastai.vision import *
from fastai import *

path = Path('D:\College\BE\BEPROJECT\Dataset2')
img = 'D:\\College\\BE\\BEPROJECT\\Normal\\00001841_000.png'

In [ ]:
learn = load_learner(path)

In [ ]:
cat , pred , probs = learn.predict(open_image(img))
if cat==tensor(0):
    print('Covid +ve ',probs[0].item())
    
else:
    print('Covid -ve ',probs[1].item())